In [2]:
import pandas as pd
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
auction = pd.read_csv("data/auction.csv")
# brand = pd.read_csv("data/brand.csv")
# category = pd.read_csv("data/category.csv")
# danjobetsu = pd.read_csv("data/danjobetsu.csv")
# genre = pd.read_csv("data/genre.csv")
# genregroup = pd.read_csv("data/genregroup.csv")
# itemshou = pd.read_csv("data/itemshou.csv")
# kaiin = pd.read_csv("data/kaiin.csv")
# line = pd.read_csv("data/line.csv")
# nyuukaoshirase = pd.read_csv("data/nyuuka_oshirase.csv")
rakusatsu = pd.read_csv("data/rakusatsu.csv")
# search_log=pd.read_csv("data/search_log.tsv")
shudounyuusatsu = pd.read_csv("data/shudounyuusatsu.csv")
watchlist = pd.read_csv("data/watchlist.csv")

submission = pd.read_csv("data/sample_submission.csv")
submission = submission.KaiinID.unique()
submission = pd.DataFrame([list(submission)], index=["KaiinID"]).T

auction["CreateDateOrder"] = auction.CreateDate.map(lambda x: x[0:10]).map(
    lambda x: datetime.strptime(x, '%Y-%m-%d'))-datetime.strptime('2019-10-01', '%Y-%m-%d')
auction["CreateDateOrder"] = auction.CreateDateOrder.map(
    lambda x: x.days).map(int)

watchlist["TourokuDateOrder"] = watchlist.TourokuDate.map(lambda x: x[0:10]).map(
    lambda x: datetime.strptime(x, '%Y-%m-%d'))-datetime.strptime('2019-10-01', '%Y-%m-%d')
watchlist["TourokuDateOrder"] = watchlist["TourokuDateOrder"].map(
    lambda x: x.days).map(int)

watchlist = pd.merge(watchlist, auction, on="AuctionID")

shudounyuusatsu["ShudouNyuusatsuDateOrder"] = shudounyuusatsu.ShudouNyuusatsuDate.map(lambda x: x[0:10]).map(
    lambda x: datetime.strptime(x, '%Y-%m-%d'))-datetime.strptime('2019-10-01', '%Y-%m-%d')
shudounyuusatsu["ShudouNyuusatsuDateOrder"] = shudounyuusatsu["ShudouNyuusatsuDateOrder"].map(
    lambda x: x.days).map(int)
shudounyuusatsu = pd.merge(shudounyuusatsu, auction, on='AuctionID')

rakusatsu["RakusatsuDateOrder"] = rakusatsu.RakusatsuDate.map(lambda x: x[0:10]).map(
    lambda x: datetime.strptime(x, '%Y-%m-%d'))-datetime.strptime('2019-10-01', '%Y-%m-%d')
rakusatsu["RakusatsuDateOrder"] = rakusatsu["RakusatsuDateOrder"].map(
    lambda x: x.days).map(int)
rakusatsu = pd.merge(rakusatsu, submission, on='KaiinID')
rakusatsu = pd.merge(rakusatsu, auction, on='AuctionID')

i = 1
dateorder = -7
inputdateorder = dateorder-7*i

In [3]:
submission

,KaiinID
0,24
1,40
2,63
3,77
4,154
...,...
6011,277760
6012,277779
6013,277803
6014,277848


In [13]:
def createfeature(inputsubmission, inputwatchlist, inputshudounyuusatsu,
                  inputrwatchlistsatsu, inputdateorder, inputauction):
    df = pd.concat([inputwatchlist[["KaiinID", "ShouhinID"]],
                    inputshudounyuusatsu[["KaiinID", "ShouhinID"]]])
    df = df[df.duplicated() != True]
    df = pd.merge(df, inputauction[["AuctionID", "ShouhinID"]], on='ShouhinID')

    df = df[df.duplicated() != True]
    df = pd.merge(df, inputsubmission, on="KaiinID")

    inputauction['CreateDateOrder'] = inputdateorder - \
        inputauction['CreateDateOrder']
    df = pd.merge(df, auction[[
        "AuctionID", "ShouhinShubetsuID", "ShouhinID", "SaishuppinKaisuu",
        "ConditionID", "BrandID", "GenreID", "GenreGroupID", "LineID", "ColorID", "DanjobetsuID", ]])

    inputwatchlist.TourokuDateOrder = inputdateorder - inputwatchlist.TourokuDateOrder
    df = pd.merge(df, inputwatchlist.groupby(["KaiinID", "AuctionID"]).TourokuDateOrder.max().reset_index().rename(
        columns={'TourokuDateOrder': 'TourokuDateOrdermax'}), how='left')

    df = pd.merge(df, inputwatchlist.groupby(["KaiinID", "AuctionID"]).TourokuDateOrder.min().reset_index().rename(
        columns={'TourokuDateOrder': 'TourokuDateOrdermin'}), how='left')

    df = pd.merge(df, inputwatchlist.groupby(["KaiinID", "AuctionID"]).TourokuDateOrder.count().reset_index().rename(
        columns={'TourokuDateOrder': 'TourokuCount'}), how='left')

    df["TourokuDatemaxmindiff"] = df.TourokuDateOrdermax - df.TourokuDateOrdermin

    inputwatchlist.groupby(["KaiinID", "AuctionID"]).SakujoFlag.max(
    ).reset_index().rename(columns={'SakujoFlag': 'watchSakujo'})

    df['TourokuDateOrderminsin'] = df['TourokuDateOrdermin'].map(np.sin)

    df['TourokuDateOrdermincos'] = df['TourokuDateOrdermin'].map(np.cos)

    Shouhincount = pd.concat([pd.Series(list(inputwatchlist.groupby("ShouhinID").ShouhinID.count().index)),
                              pd.Series(inputwatchlist.groupby("ShouhinID").ShouhinID.count().values)], axis=1)
    Shouhincount.columns = ["ShouhinID", "ShouhinWatchedCount"]
    df = pd.merge(df, Shouhincount, how="left")

    Auctioncount = pd.concat([pd.Series(list(inputwatchlist.groupby("AuctionID").AuctionID.count().index)),
                              pd.Series(inputwatchlist.groupby("AuctionID").AuctionID.count().values)], axis=1)
    Auctioncount.columns = ["AuctionID", "AuctionWatchedCount"]
    Auctioncount
    df = pd.merge(df, Auctioncount, how="left")

    Brandcount = pd.concat([pd.Series(list(inputwatchlist.groupby("BrandID").BrandID.count().index)),
                            pd.Series(inputwatchlist.groupby("BrandID").BrandID.count().values)], axis=1)
    Brandcount.columns = ["BrandID", "BrandWatchedCount"]
    Brandcount
    df = pd.merge(df, Brandcount, how="left")

    Linecount = pd.concat([
        pd.Series(list(inputwatchlist.groupby("LineID").LineID.count().index)),
        pd.Series(inputwatchlist.groupby("LineID").LineID.count().values)
    ],
        axis=1)
    Linecount.columns = ["LineID", "LineWatchedCount"]
    df = pd.merge(df, Linecount, how="left")

    ShouhinKaiinWatchcount = inputwatchlist.groupby(
        ["KaiinID", "ShouhinID"]).AuctionID.count()
    ShouhinKaiinWatchcount = ShouhinKaiinWatchcount.reset_index().rename(
        columns={"AuctionID": "ShouhinKaiinWatchCount"})
    df = pd.merge(df, ShouhinKaiinWatchcount, how="left")

    BrandKaiinWatchcount = inputwatchlist.groupby(
        ["KaiinID", "BrandID"]).AuctionID.count()
    BrandKaiinWatchcount = BrandKaiinWatchcount.reset_index().rename(
        columns={"AuctionID": "BrandKaiinWatchCount"})
    df = pd.merge(df, BrandKaiinWatchcount, how="left")

    #     GenreKaiinWatchcount = inputwatchlist.groupby(
    #         ["KaiinID", "GenreID"]).AuctionID.count()
    #     GenreKaiinWatchcount = GenreKaiinWatchcount.reset_index().rename(
    #         columns={"AuctionID": "GenreKaiinWatchCount"})
    #     df = pd.merge(df, GenreKaiinWatchcount, how="left")

    GenreGroupKaiinWatchcount = inputwatchlist.groupby(
        ["KaiinID", "GenreGroupID"]).AuctionID.count()
    GenreGroupKaiinWatchcount = GenreGroupKaiinWatchcount.reset_index().rename(
        columns={"AuctionID": "GenreGroupKaiinWatchCount"})
    df = pd.merge(df, GenreGroupKaiinWatchcount, how="left")

    LineKaiinWatchcount = inputwatchlist.groupby(
        ["KaiinID", "LineID"]).AuctionID.count()
    LineKaiinWatchcount = LineKaiinWatchcount.reset_index().rename(
        columns={"AuctionID": "LineKaiinWatchCount"})
    df = pd.merge(df, LineKaiinWatchcount, how="left")

    inputshudounyuusatsu.ShudouNyuusatsuDateOrder = inputdateorder - \
        inputshudounyuusatsu.ShudouNyuusatsuDateOrder
    df = pd.merge(df, inputshudounyuusatsu.groupby(["KaiinID", "AuctionID"]).ShudouNyuusatsuDateOrder.max().reset_index().rename(
        columns={'ShudouNyuusatsuDateOrder': 'ShudouNyuusatsuDateOrdermax'}), how='left')

    df = pd.merge(df, inputshudounyuusatsu.groupby(["KaiinID", "AuctionID"]).ShudouNyuusatsuDateOrder.min().reset_index().rename(
        columns={'ShudouNyuusatsuDateOrder': 'ShudouNyuusatsuDateOrdermin'}), how='left')

    df = pd.merge(df, inputshudounyuusatsu.groupby(["KaiinID", "AuctionID"]).ShudouNyuusatsuDateOrder.count().reset_index().rename(
        columns={'ShudouNyuusatsuDateOrder': 'ShudouNyuusatsuCount'}), how='left')

    df["ShudouNyuusatsuDatemaxmindiff"] = df.ShudouNyuusatsuDateOrdermax - \
        df.ShudouNyuusatsuDateOrdermin

    df['ShudouNyuusatsuDateOrderminsin'] = df['ShudouNyuusatsuDateOrdermin'].map(
        np.sin)

    df['ShudouNyuusatsuDateOrdermincos'] = df['ShudouNyuusatsuDateOrdermin'].map(
        np.cos)

    AuctionNyuusatsucount = pd.concat([pd.Series(list(inputshudounyuusatsu.groupby("AuctionID").AuctionID.count().index)),
                                       pd.Series(inputshudounyuusatsu.groupby("AuctionID").AuctionID.count().values)], axis=1)
    AuctionNyuusatsucount.columns = ["AuctionID", "AuctionNyuudstsuCount"]
    df = pd.merge(df, AuctionNyuusatsucount, how="left")

    ShouhinNyuusatsucount = pd.concat([pd.Series(list(inputshudounyuusatsu.groupby("ShouhinID").ShouhinID.count().index)),
                                       pd.Series(inputshudounyuusatsu.groupby("ShouhinID").AuctionID.count().values)], axis=1)
    ShouhinNyuusatsucount.columns = ["ShouhinID", "ShouhinNyuusatsuCount"]
    df = pd.merge(df, ShouhinNyuusatsucount, how="left")

    KaiinNyuusatsucount = pd.concat([pd.Series(list(inputshudounyuusatsu.groupby("KaiinID").KaiinID.count().index)),
                                     pd.Series(inputshudounyuusatsu.groupby("KaiinID").KaiinID.count().values)], axis=1)
    KaiinNyuusatsucount.columns = ["KaiinID", "KaiinNyuudstsuCount"]
    df = pd.merge(df, KaiinNyuusatsucount, how="left")

    BrandNyuusatsucount = pd.concat([
        pd.Series(list(inputshudounyuusatsu.groupby(
            "BrandID").BrandID.count().index)),
        pd.Series(inputshudounyuusatsu.groupby("BrandID").BrandID.count().values)], axis=1)
    BrandNyuusatsucount.columns = ["BrandID", "BrandNyuudstsuCount"]
    df = pd.merge(df, BrandNyuusatsucount, how="left")

#     BrandNyuusatsucount = pd.concat([
#         pd.Series(list(inputshudounyuusatsu.groupby("BrandID").BrandID.count().index)),
#         pd.Series(inputshudounyuusatsu.groupby("BrandID").BrandID.count().values)],axis=1)
#     BrandNyuusatsucount.columns = ["BrandID", "BrandNyuudstsuCount"]
#     df = pd.merge(df, BrandNyuusatsucount, how="left")

    LineNyuusatsucount = pd.concat([
        pd.Series(list(inputshudounyuusatsu.groupby(
            "LineID").LineID.count().index)),
        pd.Series(inputshudounyuusatsu.groupby("LineID").LineID.count().values)], axis=1)
    LineNyuusatsucount.columns = ["LineID", "LineNyuudstsuCount"]
    df = pd.merge(df, LineNyuusatsucount, how="left")

    ShouhinKaiinNyuusatsucount = inputshudounyuusatsu.groupby(
        ["KaiinID", "ShouhinID"]).AuctionID.count()
    ShouhinKaiinNyuusatsucount = ShouhinKaiinNyuusatsucount.reset_index(
    ).rename(columns={"AuctionID": "ShouhinKaiinNyuusatsuCount"})
    df = pd.merge(df, ShouhinKaiinNyuusatsucount, how="left")

    BrandKaiinNyuusatsucount = inputshudounyuusatsu.groupby(
        ["KaiinID", "BrandID"]).AuctionID.count()
    BrandKaiinNyuusatsucount = BrandKaiinNyuusatsucount.reset_index().rename(
        columns={"AuctionID": "BrandKaiinNyuusatsuCount"})
    df = pd.merge(df, BrandKaiinNyuusatsucount, how="left")

#     GenreKaiinNyuusatsucount = inputshudounyuusatsu.groupby(
#         ["KaiinID", "GenreID"]).AuctionID.count()
#     GenreKaiinNyuusatsucount = GenreKaiinNyuusatsucount.reset_index().rename(
#         columns={"AuctionID": "GenreKaiinNyuusatuCount"})
#     df = pd.merge(df, GenreKaiinNyuusatsucount, how="left")

    GenreGroupKaiinNyuusatsucount = inputshudounyuusatsu.groupby(
        ["KaiinID", "GenreGroupID"]).AuctionID.count()
    GenreGroupKaiinNyuusatsucount = GenreGroupKaiinNyuusatsucount.reset_index().rename(
        columns={"AuctionID": "GenreGroupKaiinNyuusatuCount"})
    df = pd.merge(df, GenreGroupKaiinNyuusatsucount, how="left")

    LineKaiinNyuusatsucount = inputshudounyuusatsu.groupby(
        ["KaiinID", "LineID"]).AuctionID.count()
    LineKaiinNyuusatsucount = LineKaiinNyuusatsucount.reset_index().rename(
        columns={"AuctionID": "LineKaiinNyuusatuCount"})
    df = pd.merge(df, LineKaiinNyuusatsucount, how="left")

    df = df.drop(["ShouhinID", "ShouhinShubetsuID", "BrandID",
                  "GenreID", "GenreGroupID", "LineID"], axis=1)

    df = df.fillna(-1)

    return df.sort_values(["KaiinID","AuctionID"]).reset_index(drop=True)

In [14]:
from tqdm import tqdm
import numpy as np
train_df=None
i=1
dateorder = -7


for i in tqdm(range(1,5)):
    # watchlist.TourokuDateOrder=dateorder - watchlist.TourokuDateOrder
    inputdateorder= dateorder-7*i
    train_watch=watchlist[(watchlist.TourokuDateOrder>=dateorder-7*i)&(watchlist.TourokuDateOrder<dateorder-7*i+7)]
    train_watch["tag"]=1
    train_watch=train_watch[["KaiinID","AuctionID","tag"]]

    train_nyuusatsu=shudounyuusatsu[(shudounyuusatsu.ShudouNyuusatsuDateOrder>=dateorder-7*i)&(shudounyuusatsu.ShudouNyuusatsuDateOrder<dateorder-7*i+7)]
    train_nyuusatsu["tag"]=2
    train_nyuusatsu=train_nyuusatsu[["KaiinID","AuctionID","tag"]]
    train_ref_df=pd.concat([train_nyuusatsu,train_watch])
    # .groupby(["KaiinID","AuctionID"]).max().reset_index()
    train_KaiinID=pd.Series(train_ref_df.KaiinID.unique())
    train_KaiinID.name="KaiinID"
    train_KaiinID=pd.DataFrame(train_KaiinID)
    train_watchlist=watchlist[watchlist.TourokuDateOrder < dateorder-7*i]
    train_shudounyuusatsu=shudounyuusatsu[shudounyuusatsu.ShudouNyuusatsuDateOrder<dateorder-7*i]
    train_rakusatsu=rakusatsu[rakusatsu.RakusatsuDateOrder<dateorder-7*i]

    train_df_child=createfeature(train_KaiinID, train_watchlist, train_shudounyuusatsu, train_rakusatsu, dateorder-7*i,auction)
    watch_kari=watchlist[(watchlist.TourokuDateOrder>=dateorder-7*i)&(watchlist.TourokuDateOrder<dateorder-7*i+7)]
    watch_kari["watchtag"]=1
    watch_kari=watch_kari[["KaiinID","AuctionID","watchtag"]]
    watch_kari=watch_kari[~watch_kari.duplicated()]
    train_df_child=pd.merge(train_df_child,watch_kari,how="left")
    shudounyuusatsu_kari=shudounyuusatsu[(shudounyuusatsu.ShudouNyuusatsuDateOrder>=dateorder-7*i)&(shudounyuusatsu.ShudouNyuusatsuDateOrder<dateorder-7*i+7)]
    shudounyuusatsu_kari["nyuusatsutag"]=1
    shudounyuusatsu_kari=shudounyuusatsu_kari[["KaiinID","AuctionID","nyuusatsutag"]]
    shudounyuusatsu_kari=shudounyuusatsu_kari[~shudounyuusatsu_kari.duplicated()]
    train_df_child=pd.merge(train_df_child,shudounyuusatsu_kari,how="left")
    train_df_child=train_df_child.fillna(0)

    train_df=pd.concat([train_df,train_df_child]).sort_values(["KaiinID","AuctionID"]).reset_index(drop=True)

100%|██████████| 4/4 [08:39<00:00, 129.91s/it]


In [32]:
auction

,AuctionID,ShouhinShubetsuID,ShouhinID,SaishuppinKaisuu,ConditionID,BrandID,GenreID,GenreGroupID,LineID,ColorID,DanjobetsuID,SankouKakaku,CreateDate,CreateDateOrder
0,4066465,1,620477,334,6,494,321,312,0,15,0,4000,2019-07-27 09:31:34,-59
1,622037,1,1146271,0,6,32,321,312,0,304,0,4500,2019-07-27 09:31:34,-59
2,3447030,1,1069454,0,6,30,320,312,0,338,0,3000,2019-07-27 09:31:34,-59
3,957658,1,661292,0,6,30,321,312,0,188,0,3500,2019-07-27 09:31:34,-59
4,2496085,1,2207903,0,6,569,6,314,0,0,0,8000,2019-07-27 09:31:34,-59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4109554,91078,1,2369558,0,7,41,16,16,0,0,0,3000,2018-11-06 09:43:21,-322
4109555,4089381,1,1147062,0,7,3471,321,312,0,0,0,10000,2018-11-06 09:43:21,-322
4109556,2655736,1,688258,0,6,569,341,312,0,290,0,16000,2018-11-06 09:43:21,-322
4109557,1889335,1,1289629,0,4,569,341,312,0,0,0,6000,2018-11-06 09:43:21,-322


In [34]:
import lightgbm as lgb
watch_train = lgb.Dataset(train_df.drop(
    ["KaiinID", "AuctionID", "watchtag", "nyuusatsutag"], axis=1), train_df.watchtag)
nyuusatsu_train = lgb.Dataset(train_df.drop(
    ["KaiinID", "AuctionID", "watchtag", "nyuusatsutag"], axis=1), train_df.nyuusatsutag)

# LightGBM のハイパーパラメータ
lgbm_params = {
    # 二値分類問題
    'objective': 'binary',
    # AUC の最大化を目指す
    "nround": 500,
    "learning_rate": 0.01,
    "max_depth": 6,
    "num_leaves": 127,
    "verbosity": -1,
    "random_seed": 0,
}
for i in tqdm(range(2)):
    if i==0:
        watch_model = lgb.train(lgbm_params, watch_train)
    else:
        nyuusatsu_model = lgb.train(lgbm_params, nyuusatsu_train)

test_df = createfeature(submission, watchlist,
                        shudounyuusatsu, rakusatsu, dateorder,auction)

test_df["watchtag"] = -1
test_df["nyuusatsutag"] = -1

watch_point=watch_model.predict(test_df.drop(
    ["KaiinID", "AuctionID", "watchtag", "nyuusatsutag"], axis=1))

nyuusatsu_point=nyuusatsu_model.predict(test_df.drop(
    ["KaiinID", "AuctionID", "watchtag", "nyuusatsutag"], axis=1))

predict = test_df[["KaiinID", "AuctionID"]]
predict["Point"] = 0.4*watch_point+0.6*nyuusatsu_point

kari = auction[auction.CreateDateOrder == dateorder-5].sample(n=20, random_state=0)
addtionaltable = kari["AuctionID"]

num=submission.T.values[0]


for i in range(len(num)):
    
    addtionaltable=pd.DataFrame({"KaiinID":pd.Series([num[i]]*20),"AuctionID":kari["AuctionID"].reset_index(drop=True),"Point":pd.Series([0]*20)})
    
    sub1=pd.concat([pd.concat([predict[predict.KaiinID==num[i]],addtionaltable]).reset_index(drop=True)[["KaiinID","AuctionID"]],
           pd.concat([predict[predict.KaiinID==num[i]],addtionaltable]).Point.rank(method='first',ascending=False).reset_index(drop=True)],axis=1)
# .rank(method='first',ascending=False).Point
    if i==0:
        sub=sub1[(0<sub1.Point)&(sub1.Point<21)]
    else:
        sub=pd.concat([sub,sub1[(0<sub1.Point)&(sub1.Point<21)]])
    if i%100==0:
        print(str(i)+"回目")
sub[["KaiinID","AuctionID"]].to_csv("submission.csv",index = False)

0回目
100回目
200回目
300回目
400回目
500回目
600回目
700回目
800回目
900回目
1000回目
1100回目
1200回目
1300回目
1400回目
1500回目
1600回目
1700回目
1800回目
1900回目
2000回目
2100回目
2200回目
2300回目
2400回目
2500回目
2600回目
2700回目
2800回目
2900回目
3000回目
3100回目
3200回目
3300回目
3400回目
3500回目
3600回目
3700回目
3800回目
3900回目
4000回目
4100回目
4200回目
4300回目
4400回目
4500回目
4600回目
4700回目
4800回目
4900回目
5000回目
5100回目
5200回目
5300回目
5400回目
5500回目
5600回目
5700回目
5800回目
5900回目
6000回目


In [16]:
train_df.tail(50)

,KaiinID,AuctionID,SaishuppinKaisuu,ConditionID,ColorID,DanjobetsuID,TourokuDateOrdermax,TourokuDateOrdermin,TourokuCount,TourokuDatemaxmindiff,...,ShouhinNyuusatsuCount,KaiinNyuudstsuCount,BrandNyuudstsuCount,LineNyuudstsuCount,ShouhinKaiinNyuusatsuCount,BrandKaiinNyuusatsuCount,GenreGroupKaiinNyuusatuCount,LineKaiinNyuusatuCount,watchtag,nyuusatsutag
13310407,277891,4055075,0,6,0,2,-1.0,-1.0,-1.0,-1.0,...,-1.0,91.0,20.0,539407.0,-1.0,-1.0,29.0,91.0,0.0,0.0
13310408,277891,4055075,0,6,0,2,-1.0,-1.0,-1.0,-1.0,...,-1.0,87.0,20.0,529031.0,-1.0,-1.0,29.0,87.0,0.0,0.0
13310409,277891,4059566,0,6,0,2,3.0,3.0,1.0,0.0,...,-1.0,92.0,329.0,560636.0,-1.0,-1.0,29.0,92.0,0.0,0.0
13310410,277891,4061278,0,6,323,2,328.0,328.0,1.0,0.0,...,-1.0,92.0,135.0,560636.0,-1.0,-1.0,8.0,92.0,0.0,0.0
13310411,277891,4061278,0,6,323,2,321.0,321.0,1.0,0.0,...,-1.0,92.0,134.0,549648.0,-1.0,-1.0,8.0,92.0,0.0,0.0
13310412,277891,4061278,0,6,323,2,314.0,314.0,1.0,0.0,...,-1.0,91.0,134.0,539407.0,-1.0,-1.0,8.0,91.0,0.0,0.0
13310413,277891,4061278,0,6,323,2,307.0,307.0,1.0,0.0,...,-1.0,87.0,133.0,529031.0,-1.0,-1.0,5.0,87.0,0.0,0.0
13310414,277891,4067762,0,6,0,0,290.0,290.0,1.0,0.0,...,-1.0,92.0,19.0,560636.0,-1.0,-1.0,18.0,92.0,0.0,0.0
13310415,277891,4067762,0,6,0,0,283.0,283.0,1.0,0.0,...,-1.0,92.0,19.0,549648.0,-1.0,-1.0,18.0,92.0,0.0,0.0
13310416,277891,4067762,0,6,0,0,276.0,276.0,1.0,0.0,...,-1.0,91.0,19.0,539407.0,-1.0,-1.0,17.0,91.0,0.0,0.0


In [83]:
sub[["KaiinID","AuctionID"]].to_csv("submission.csv",index = False)

In [33]:
auction[auction.CreateDateOrder == dateorder-5]

,AuctionID,ShouhinShubetsuID,ShouhinID,SaishuppinKaisuu,ConditionID,BrandID,GenreID,GenreGroupID,LineID,ColorID,DanjobetsuID,SankouKakaku,CreateDate,CreateDateOrder
1895505,3510091,1,2262826,0,6,772,341,312,0,188,0,1580,2019-09-12 09:30:02,-12
1895506,3592300,1,2883186,0,6,5340,320,312,0,175,0,1000,2019-09-12 09:30:02,-12
1895507,3785373,1,2152844,0,7,8220,320,312,0,0,0,1360,2019-09-12 09:30:02,-12
1895508,3868958,1,2071126,0,6,742,327,312,0,0,0,1000,2019-09-12 09:30:03,-12
1895509,2216612,1,417404,0,6,1085,321,312,0,15,0,15000,2019-09-12 09:30:03,-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1903143,3912094,1,2670693,0,7,70,320,312,0,314,0,60000,2019-09-12 09:35:26,-12
1903144,459664,1,288811,0,6,38,341,312,0,15,0,48000,2019-09-12 09:35:26,-12
1903145,232054,1,160992,0,7,25,321,312,0,15,0,73000,2019-09-12 09:35:26,-12
1903146,3530530,1,531617,351,7,70,341,312,0,0,0,40000,2019-09-12 09:35:26,-12


In [21]:
from tqdm import tqdm
train_df=None
i=1
dateorder = -7
for i in tqdm(range(1,3)):
    print(i)
    train_watch=watchlist[(watchlist.TourokuDateOrder>=dateorder-7*i)&(watchlist.TourokuDateOrder<dateorder-7*i+7)]
    train_watch["tag"]=1
    train_watch=train_watch[["KaiinID","AuctionID","tag"]]

    train_nyuusatsu=shudounyuusatsu[(shudounyuusatsu.ShudouNyuusatsuDateOrder>=dateorder-7*i)&(shudounyuusatsu.ShudouNyuusatsuDateOrder<dateorder-7*i+7)]
    train_nyuusatsu["tag"]=2
    train_nyuusatsu=train_nyuusatsu[["KaiinID","AuctionID","tag"]]

    train_ref_df=pd.concat([train_nyuusatsu,train_watch]).groupby(["KaiinID","AuctionID"]).max().reset_index()

    train_KaiinID=pd.Series(train_ref_df.KaiinID.unique())
    train_KaiinID.name="KaiinID"
    train_KaiinID=pd.DataFrame(train_KaiinID)

    train_watchlist=watchlist[watchlist.TourokuDateOrder < dateorder-7*i]

    train_shudounyuusatsu=shudounyuusatsu[shudounyuusatsu.ShudouNyuusatsuDateOrder<dateorder-7*i]

    train_rakusatsu=rakusatsu[rakusatsu.RakusatsuDateOrder<dateorder-7*i]

    train_df_child=createfeature(train_KaiinID, train_watchlist, train_shudounyuusatsu, train_rakusatsu, dateorder-7*i,auction)

    watch_kari=watchlist[(watchlist.TourokuDateOrder>=dateorder-7*i)&(watchlist.TourokuDateOrder<dateorder-7*i+7)]
    watch_kari["watchtag"]=1
    watch_kari=watch_kari[["KaiinID","AuctionID","watchtag"]]
    watch_kari=watch_kari[~watch_kari.duplicated()]
    train_df_child=pd.merge(train_df_child,watch_kari,how="left")
    shudounyuusatsu_kari=shudounyuusatsu[(shudounyuusatsu.ShudouNyuusatsuDateOrder>=dateorder-7*i)&(shudounyuusatsu.ShudouNyuusatsuDateOrder<dateorder-7*i+7)]
    shudounyuusatsu_kari["nyuusatsutag"]=1
    shudounyuusatsu_kari=shudounyuusatsu_kari[["KaiinID","AuctionID","nyuusatsutag"]]
    shudounyuusatsu_kari=shudounyuusatsu_kari[~shudounyuusatsu_kari.duplicated()]
    train_df_child=pd.merge(train_df_child,shudounyuusatsu_kari,how="left")
    train_df_child=train_df_child.fillna(0)

    train_df=pd.concat([train_df,train_df_child])
    train_df

  0%|          | 0/2 [00:00<?, ?it/s]

1


 50%|█████     | 1/2 [01:56<01:56, 116.04s/it]

2


100%|██████████| 2/2 [03:48<00:00, 114.22s/it]


In [22]:
train_df.tail(50)

,KaiinID,AuctionID,SaishuppinKaisuu,ConditionID,ColorID,DanjobetsuID,TourokuDateOrdermax,TourokuDateOrdermin,TourokuCount,TourokuDatemaxmindiff,...,ShouhinNyuusatsuCount,KaiinNyuudstsuCount,BrandNyuudstsuCount,LineNyuudstsuCount,ShouhinKaiinNyuusatsuCount,BrandKaiinNyuusatsuCount,GenreGroupKaiinNyuusatuCount,LineKaiinNyuusatuCount,watchtag,nyuusatsutag
3373149,277891,3898544,0,5,0,0,-1.0,-1.0,-1.0,-1.0,...,-1.0,92.0,15366.0,549648.0,-1.0,-1.0,18.0,92.0,0.0,0.0
3373150,277891,3898764,0,7,144,2,8.0,8.0,1.0,0.0,...,1.0,92.0,377.0,549648.0,1.0,1.0,8.0,92.0,0.0,0.0
3373151,277891,3898863,0,5,0,0,-1.0,-1.0,-1.0,-1.0,...,-1.0,92.0,15366.0,549648.0,-1.0,-1.0,18.0,92.0,0.0,0.0
3373152,277891,3900680,0,6,0,2,-1.0,-1.0,-1.0,-1.0,...,1.0,92.0,117.0,549648.0,1.0,2.0,8.0,92.0,0.0,0.0
3373153,277891,3900874,0,6,46,0,-1.0,-1.0,-1.0,-1.0,...,-1.0,92.0,15366.0,549648.0,-1.0,-1.0,18.0,92.0,0.0,0.0
3373154,277891,3901333,0,6,0,2,-1.0,-1.0,-1.0,-1.0,...,1.0,92.0,276.0,549648.0,1.0,1.0,29.0,92.0,0.0,0.0
3373155,277891,3902652,0,6,58,2,325.0,325.0,1.0,0.0,...,2.0,92.0,110.0,549648.0,-1.0,-1.0,-1.0,92.0,0.0,0.0
3373156,277891,3904426,250,6,338,0,13.0,13.0,1.0,0.0,...,-1.0,92.0,15366.0,549648.0,-1.0,-1.0,18.0,92.0,0.0,0.0
3373157,277891,3921625,0,6,0,0,-1.0,-1.0,-1.0,-1.0,...,1.0,92.0,11.0,549648.0,1.0,1.0,18.0,92.0,0.0,0.0
3373158,277891,3925149,0,8,58,2,108.0,108.0,1.0,0.0,...,-1.0,92.0,9.0,549648.0,-1.0,-1.0,-1.0,92.0,0.0,0.0


In [23]:
watch_kari=watchlist[(watchlist.TourokuDateOrder>=dateorder-7*i)&(watchlist.TourokuDateOrder<dateorder-7*i+7)]
watch_kari["watchtag"]=1
watch_kari=watch_kari[["KaiinID","AuctionID","watchtag"]]
watch_kari=watch_kari[~watch_kari.duplicated()]
train_df_child=pd.merge(train_df_child,watch_kari,how="left")

In [24]:
train_df_child

,KaiinID,AuctionID,SaishuppinKaisuu,ConditionID,ColorID,DanjobetsuID,TourokuDateOrdermax,TourokuDateOrdermin,TourokuCount,TourokuDatemaxmindiff,...,AuctionNyuudstsuCount,ShouhinNyuusatsuCount,KaiinNyuudstsuCount,BrandNyuudstsuCount,LineNyuudstsuCount,ShouhinKaiinNyuusatsuCount,BrandKaiinNyuusatsuCount,GenreGroupKaiinNyuusatuCount,LineKaiinNyuusatuCount,watchtag
0,231713,1036190,0,6,15,0,381.0,381.0,1.0,0.0,...,-1.0,-1.0,45.0,317.0,560636.0,-1.0,-1.0,-1.0,41.0,NaN
1,231713,4106265,0,6,68,0,353.0,353.0,1.0,0.0,...,-1.0,-1.0,45.0,4438.0,560636.0,-1.0,-1.0,1.0,41.0,NaN
2,58564,4106265,0,6,68,0,358.0,358.0,1.0,0.0,...,-1.0,-1.0,54.0,4438.0,560636.0,-1.0,-1.0,2.0,54.0,NaN
3,81233,4106265,0,6,68,0,381.0,381.0,1.0,0.0,...,-1.0,-1.0,47.0,4438.0,560636.0,-1.0,-1.0,-1.0,47.0,NaN
4,79459,4106265,0,6,68,0,348.0,348.0,1.0,0.0,...,-1.0,-1.0,24.0,4438.0,560636.0,-1.0,7.0,-1.0,24.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3445772,223521,2810503,0,6,0,0,-1.0,-1.0,-1.0,-1.0,...,-1.0,1.0,1.0,7.0,560636.0,1.0,1.0,1.0,1.0,NaN
3445773,223521,1480392,0,6,0,0,-1.0,-1.0,-1.0,-1.0,...,-1.0,1.0,1.0,7.0,560636.0,1.0,1.0,1.0,1.0,NaN
3445774,223521,2670120,0,6,0,0,-1.0,-1.0,-1.0,-1.0,...,-1.0,1.0,1.0,7.0,560636.0,1.0,1.0,1.0,1.0,NaN
3445775,159619,4029340,0,6,68,0,-1.0,-1.0,-1.0,-1.0,...,1.0,1.0,2.0,4576.0,560636.0,1.0,1.0,1.0,2.0,NaN


In [37]:
inputsubmission, inputwatchlist, inputshudounyuusatsu,inputrakusatsu, inputdateorder=submission,watchlist,shudounyuusatsu,rakusatsu,inputdateorder
df = pd.concat([inputwatchlist[["KaiinID", "ShouhinID"]],inputshudounyuusatsu[["KaiinID", "ShouhinID"]]])
df = df[df.duplicated() != True]

# df = pd.merge(df, auction[["AuctionID", "ShouhinID"]], on='ShouhinID')
# df = df[df.duplicated() != True][["KaiinID","AuctionID"]]

# df = pd.merge(df, inputsubmission, on="KaiinID")
# auction['CreateDateOrder'] = inputdateorder - auction['CreateDateOrder']
# df = pd.merge(df, auction[[
#     "AuctionID","ShouhinShubetsuID","ShouhinID","SaishuppinKaisuu",
#     "ConditionID","BrandID","GenreID","GenreGroupID","LineID","ColorID","DanjobetsuID",]])

# inputwatchlist.TourokuDateOrder = inputdateorder - inputwatchlist.TourokuDateOrder
# df = pd.merge(df,inputwatchlist.groupby(["KaiinID", "AuctionID"]).TourokuDateOrder.max().reset_index().rename(
#                   columns={'TourokuDateOrder': 'TourokuDateOrdermax'}),how='left')

# df = pd.merge(df,inputwatchlist.groupby(["KaiinID", "AuctionID"]).TourokuDateOrder.min().reset_index().rename(
#                   columns={'TourokuDateOrder': 'TourokuDateOrdermin'}),how='left')

# df = pd.merge(df,inputwatchlist.groupby(["KaiinID", "AuctionID"]).TourokuDateOrder.count().reset_index().rename(
#                   columns={'TourokuDateOrder': 'TourokuCount'}),how='left')

# df["TourokuDatemaxmindiff"] = df.TourokuDateOrdermax - df.TourokuDateOrdermin

# df = pd.merge(df,inputwatchlist.groupby(["KaiinID","AuctionID"]).SakujoFlag.max().reset_index().rename(
#                        columns={'SakujoFlag': 'watchSakujo'}),how='left')

# Shouhincount = pd.concat([pd.Series(list(inputwatchlist.groupby("ShouhinID").ShouhinID.count().index)),
#     pd.Series(inputwatchlist.groupby("ShouhinID").ShouhinID.count().values)],axis=1)
# Shouhincount.columns = ["ShouhinID", "ShouhinWatchedCount"]
# df = pd.merge(df, Shouhincount, how="left")

# Auctioncount = pd.concat([pd.Series(list(inputwatchlist.groupby("AuctionID").AuctionID.count().index)),
#     pd.Series(inputwatchlist.groupby("AuctionID").AuctionID.count().values)],axis=1)
# Auctioncount.columns = ["AuctionID", "AuctionWatchedCount"]
# Auctioncount
# df = pd.merge(df, Auctioncount, how="left")

# Brandcount = pd.concat([pd.Series(list(inputwatchlist.groupby("BrandID").BrandID.count().index)),
#                         pd.Series(inputwatchlist.groupby("BrandID").BrandID.count().values)],axis=1)
# Brandcount.columns = ["BrandID", "BrandWatchedCount"]
# Brandcount
# df = pd.merge(df, Brandcount, how="left")

# Linecount = pd.concat([
#     pd.Series(list(inputwatchlist.groupby("LineID").LineID.count().index)),
#     pd.Series(inputwatchlist.groupby("LineID").LineID.count().values)
# ],
#     axis=1)
# Linecount.columns = ["LineID", "LineWatchedCount"]
# df = pd.merge(df, Linecount, how="left")

# ShouhinKaiinWatchcount = inputwatchlist.groupby(["KaiinID", "ShouhinID"]).AuctionID.count()
# ShouhinKaiinWatchcount = ShouhinKaiinWatchcount.reset_index().rename(columns={"AuctionID": "ShouhinKaiinWatchCount"})
# df = pd.merge(df, ShouhinKaiinWatchcount, how="left")

# BrandKaiinWatchcount = inputwatchlist.groupby(["KaiinID", "BrandID"]).AuctionID.count()
# BrandKaiinWatchcount = BrandKaiinWatchcount.reset_index().rename(columns={"AuctionID": "BrandKaiinWatchCount"})
# df = pd.merge(df, BrandKaiinWatchcount, how="left")

# #     GenreKaiinWatchcount = inputwatchlist.groupby(
# #         ["KaiinID", "GenreID"]).AuctionID.count()
# #     GenreKaiinWatchcount = GenreKaiinWatchcount.reset_index().rename(
# #         columns={"AuctionID": "GenreKaiinWatchCount"})
# #     df = pd.merge(df, GenreKaiinWatchcount, how="left")

# GenreGroupKaiinWatchcount = inputwatchlist.groupby(["KaiinID", "GenreGroupID"]).AuctionID.count()
# GenreGroupKaiinWatchcount = GenreGroupKaiinWatchcount.reset_index().rename(columns={"AuctionID": "GenreGroupKaiinWatchCount"})
# df = pd.merge(df, GenreGroupKaiinWatchcount, how="left")

# LineKaiinWatchcount = inputwatchlist.groupby(["KaiinID","LineID"]).AuctionID.count()
# LineKaiinWatchcount = LineKaiinWatchcount.reset_index().rename(
#     columns={"AuctionID": "LineKaiinWatchCount"})
# df = pd.merge(df, LineKaiinWatchcount, how="left")

# inputshudounyuusatsu.ShudouNyuusatsuDateOrder = inputdateorder - \
#     inputshudounyuusatsu.ShudouNyuusatsuDateOrder
# df = pd.merge(df,inputshudounyuusatsu.groupby(["KaiinID", "AuctionID"]).ShudouNyuusatsuDateOrder.max().reset_index().rename(
#         columns={'ShudouNyuusatsuDateOrder': 'ShudouNyuusatsuDateOrdermax'}),how='left')

# df = pd.merge(df,inputshudounyuusatsu.groupby(["KaiinID", "AuctionID"]).ShudouNyuusatsuDateOrder.min().reset_index().rename(
#         columns={'ShudouNyuusatsuDateOrder': 'ShudouNyuusatsuDateOrdermin'}),how='left')

# df = pd.merge(df,inputshudounyuusatsu.groupby(["KaiinID", "AuctionID"]).ShudouNyuusatsuDateOrder.count().reset_index().rename(
#         columns={'ShudouNyuusatsuDateOrder': 'ShudouNyuusatsuCount'}),how='left')
# df["ShudouNyuusatsuDatemaxmindiff"] = df.ShudouNyuusatsuDateOrdermax - df.ShudouNyuusatsuDateOrdermin

# AuctionNyuusatsucount = pd.concat([pd.Series(list(inputshudounyuusatsu.groupby("AuctionID").AuctionID.count().index)),
#     pd.Series(inputshudounyuusatsu.groupby("AuctionID").AuctionID.count().values)],axis=1)
# AuctionNyuusatsucount.columns = ["AuctionID", "AuctionNyuudstsuCount"]
# df = pd.merge(df, AuctionNyuusatsucount, how="left")

# ShouhinNyuusatsucount = pd.concat([pd.Series(list(inputshudounyuusatsu.groupby("ShouhinID").ShouhinID.count().index)),
#     pd.Series(inputshudounyuusatsu.groupby("ShouhinID").AuctionID.count().values)],axis=1)
# ShouhinNyuusatsucount.columns = ["ShouhinID", "ShouhinNyuusatsuCount"]
# df = pd.merge(df, ShouhinNyuusatsucount, how="left")

# # KaiinNyuusatsucount = pd.concat([pd.Series(list(inputshudounyuusatsu.groupby("KaiinID").KaiinID.count().index)),
# #     pd.Series(inputshudounyuusatsu.groupby("KaiinID").KaiinID.count().values)],axis=1)
# # KaiinNyuusatsucount.columns = ["KaiinID", "KaiinNyuudstsuCount"]
# # df = pd.merge(df, KaiinNyuusatsucount, how="left")

# BrandNyuusatsucount = pd.concat([
#     pd.Series(list(inputshudounyuusatsu.groupby("BrandID").BrandID.count().index)),
#     pd.Series(inputshudounyuusatsu.groupby("BrandID").BrandID.count().values)],axis=1)
# BrandNyuusatsucount.columns = ["BrandID", "BrandNyuudstsuCount"]
# df = pd.merge(df, BrandNyuusatsucount, how="left")

# #     BrandNyuusatsucount = pd.concat([
# #         pd.Series(list(inputshudounyuusatsu.groupby("BrandID").BrandID.count().index)),
# #         pd.Series(inputshudounyuusatsu.groupby("BrandID").BrandID.count().values)],axis=1)
# #     BrandNyuusatsucount.columns = ["BrandID", "BrandNyuudstsuCount"]
# #     df = pd.merge(df, BrandNyuusatsucount, how="left")

# LineNyuusatsucount = pd.concat([
#     pd.Series(list(inputshudounyuusatsu.groupby("LineID").LineID.count().index)),
#     pd.Series(inputshudounyuusatsu.groupby("LineID").LineID.count().values)],axis=1)
# LineNyuusatsucount.columns = ["LineID", "LineNyuudstsuCount"]
# df = pd.merge(df, LineNyuusatsucount, how="left")

# ShouhinKaiinNyuusatsucount = inputshudounyuusatsu.groupby(
#     ["KaiinID", "ShouhinID"]).AuctionID.count()
# ShouhinKaiinNyuusatsucount = ShouhinKaiinNyuusatsucount.reset_index(
# ).rename(columns={"AuctionID": "ShouhinKaiinNyuusatsuCount"})
# df = pd.merge(df, ShouhinKaiinNyuusatsucount, how="left")

# BrandKaiinNyuusatsucount = inputshudounyuusatsu.groupby(
#     ["KaiinID", "BrandID"]).AuctionID.count()
# BrandKaiinNyuusatsucount = BrandKaiinNyuusatsucount.reset_index().rename(
#     columns={"AuctionID": "BrandKaiinNyuusatsuCount"})
# df = pd.merge(df, BrandKaiinNyuusatsucount, how="left")

# #     GenreKaiinNyuusatsucount = inputshudounyuusatsu.groupby(
# #         ["KaiinID", "GenreID"]).AuctionID.count()
# #     GenreKaiinNyuusatsucount = GenreKaiinNyuusatsucount.reset_index().rename(
# #         columns={"AuctionID": "GenreKaiinNyuusatuCount"})
# #     df = pd.merge(df, GenreKaiinNyuusatsucount, how="left")

# GenreGroupKaiinNyuusatsucount = inputshudounyuusatsu.groupby(
#     ["KaiinID", "GenreGroupID"]).AuctionID.count()
# GenreGroupKaiinNyuusatsucount = GenreGroupKaiinNyuusatsucount.reset_index().rename(
#     columns={"AuctionID": "GenreGroupKaiinNyuusatuCount"})
# df = pd.merge(df, GenreGroupKaiinNyuusatsucount, how="left")

# LineKaiinNyuusatsucount = inputshudounyuusatsu.groupby(
#     ["KaiinID", "LineID"]).AuctionID.count()
# LineKaiinNyuusatsucount = LineKaiinNyuusatsucount.reset_index().rename(
#     columns={"AuctionID": "LineKaiinNyuusatuCount"})
# df = pd.merge(df, LineKaiinNyuusatsucount, how="left")

# df = df.drop(["ShouhinID", "ShouhinShubetsuID", "BrandID", "GenreID", "GenreGroupID","LineID"],axis=1)

# df = df.fillna(-1)



In [29]:
inputwatchlist.groupby(["KaiinID","AuctionID"]).SakujoFlag.max().reset_index().rename(
                       columns={'SakujoFlag': 'watchSakujo'})


,KaiinID,AuctionID,watchSakujo
0,22,82220,1
1,22,122841,1
2,22,139072,1
3,22,147277,0
4,22,147866,0
...,...,...,...
4058662,277891,4090315,1
4058663,277891,4097355,1
4058664,277891,4104172,1
4058665,277891,4109558,1


In [14]:
train_nyuusatsu=shudounyuusatsu[(shudounyuusatsu.ShudouNyuusatsuDateOrder>=dateorder-7*i)&(shudounyuusatsu.ShudouNyuusatsuDateOrder<dateorder-7*i+7)]
train_nyuusatsu["tag"]=2
train_nyuusatsu=train_nyuusatsu[["KaiinID","AuctionID","tag"]]

/home/ogata/.pyenv/versions/3.7.0/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [19]:
train_ref_df=pd.concat([train_nyuusatsu,train_watch]).groupby(["KaiinID","AuctionID"]).max().reset_index()

In [43]:
train_KaiinID=pd.Series(train_ref_df.KaiinID.unique())
train_KaiinID.name="KaiinID"
train_KaiinID=pd.DataFrame(train_KaiinID)

In [30]:
train_watchlist=watchlist[watchlist.TourokuDateOrder < dateorder-7*i]


In [33]:
train_shudounyuusatsu=shudounyuusatsu[shudounyuusatsu.ShudouNyuusatsuDateOrder<dateorder-7*i]


In [39]:
train_rakusatsu=rakusatsu[rakusatsu.RakusatsuDateOrder<dateorder-7*i]

In [55]:
i

1

In [73]:
# train_df_child=createfeature(train_KaiinID, train_watchlist, train_shudounyuusatsu, train_rakusatsu, dateorder-7*i)
watch_kari=watchlist[(watchlist.TourokuDateOrder>=dateorder-7*i)&(watchlist.TourokuDateOrder<dateorder-7*i+7)]
watch_kari["watchtag"]=1
watch_kari=watch_kari[["KaiinID","AuctionID","watchtag"]]
train_df_child=pd.merge(train_df_child,watch_kari,how="left")

/home/ogata/.pyenv/versions/3.7.0/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [82]:
shudounyuusatsu_kari=shudounyuusatsu[(shudounyuusatsu.ShudouNyuusatsuDateOrder>=dateorder-7*i)&(shudounyuusatsu.ShudouNyuusatsuDateOrder<dateorder-7*i+7)]
shudounyuusatsu_kari["nyuusatsutag"]=1
shudounyuusatsu_kari=shudounyuusatsu_kari[["KaiinID","AuctionID","nyuusatsutag"]]
train_df_child=pd.merge(train_df_child,shudounyuusatsu_kari,how="left")

/home/ogata/.pyenv/versions/3.7.0/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [84]:
train_df_child=train_df_child.fillna(0)


,KaiinID,AuctionID,SaishuppinKaisuu,ConditionID,ColorID,DanjobetsuID,TourokuDateOrdermax,TourokuDateOrdermin,TourokuCount,TourokuDatemaxmindiff,...,AuctionNyuudstsuCount,KaiinNyuudstsuCount,BrandNyuudstsuCount,LineNyuudstsuCount,ShouhinKaiinNyuusatsuCount,BrandKaiinNyuusatsuCount,GenreKaiinNyuusatuCount,LineKaiinNyuusatuCount,watchtag,nyuusatsutag
0,231713,1036190,0,6,15,0,-395.0,-395.0,1.0,0.0,...,-1.0,45.0,319.0,572160.0,-1.0,-1.0,-1.0,41.0,0.0,0.0
1,231713,4106265,0,6,68,0,-367.0,-367.0,1.0,0.0,...,-1.0,45.0,4521.0,572160.0,-1.0,-1.0,-1.0,41.0,0.0,0.0
2,58564,4106265,0,6,68,0,-372.0,-372.0,1.0,0.0,...,-1.0,54.0,4521.0,572160.0,-1.0,-1.0,1.0,54.0,0.0,0.0
3,81233,4106265,0,6,68,0,-395.0,-395.0,1.0,0.0,...,-1.0,47.0,4521.0,572160.0,-1.0,-1.0,-1.0,47.0,0.0,0.0
4,79459,4106265,0,6,68,0,-362.0,-362.0,1.0,0.0,...,-1.0,25.0,4521.0,572160.0,-1.0,7.0,-1.0,25.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3461870,47024,2007419,0,6,297,0,-1.0,-1.0,-1.0,-1.0,...,1.0,1.0,421.0,572160.0,1.0,1.0,1.0,1.0,0.0,1.0
3461871,124155,1784243,0,6,0,0,-1.0,-1.0,-1.0,-1.0,...,1.0,1.0,182.0,572160.0,1.0,1.0,1.0,1.0,0.0,1.0
3461872,87578,2923626,0,6,68,2,-1.0,-1.0,-1.0,-1.0,...,-1.0,1.0,45.0,572160.0,1.0,1.0,1.0,1.0,0.0,0.0
3461873,87578,1977247,0,6,68,2,-1.0,-1.0,-1.0,-1.0,...,-1.0,1.0,45.0,572160.0,1.0,1.0,1.0,1.0,0.0,0.0


In [86]:
train_df=None
train_df=pd.concat([train_df,train_df_child])

,KaiinID,AuctionID,SaishuppinKaisuu,ConditionID,ColorID,DanjobetsuID,TourokuDateOrdermax,TourokuDateOrdermin,TourokuCount,TourokuDatemaxmindiff,...,AuctionNyuudstsuCount,KaiinNyuudstsuCount,BrandNyuudstsuCount,LineNyuudstsuCount,ShouhinKaiinNyuusatsuCount,BrandKaiinNyuusatsuCount,GenreKaiinNyuusatuCount,LineKaiinNyuusatuCount,watchtag,nyuusatsutag
0,231713,1036190,0,6,15,0,-395.0,-395.0,1.0,0.0,...,-1.0,45.0,319.0,572160.0,-1.0,-1.0,-1.0,41.0,NaN,NaN
1,231713,4106265,0,6,68,0,-367.0,-367.0,1.0,0.0,...,-1.0,45.0,4521.0,572160.0,-1.0,-1.0,-1.0,41.0,NaN,NaN
2,58564,4106265,0,6,68,0,-372.0,-372.0,1.0,0.0,...,-1.0,54.0,4521.0,572160.0,-1.0,-1.0,1.0,54.0,NaN,NaN
3,81233,4106265,0,6,68,0,-395.0,-395.0,1.0,0.0,...,-1.0,47.0,4521.0,572160.0,-1.0,-1.0,-1.0,47.0,NaN,NaN
4,79459,4106265,0,6,68,0,-362.0,-362.0,1.0,0.0,...,-1.0,25.0,4521.0,572160.0,-1.0,7.0,-1.0,25.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3461870,47024,2007419,0,6,297,0,-1.0,-1.0,-1.0,-1.0,...,1.0,1.0,421.0,572160.0,1.0,1.0,1.0,1.0,NaN,1.0
3461871,124155,1784243,0,6,0,0,-1.0,-1.0,-1.0,-1.0,...,1.0,1.0,182.0,572160.0,1.0,1.0,1.0,1.0,NaN,1.0
3461872,87578,2923626,0,6,68,2,-1.0,-1.0,-1.0,-1.0,...,-1.0,1.0,45.0,572160.0,1.0,1.0,1.0,1.0,NaN,NaN
3461873,87578,1977247,0,6,68,2,-1.0,-1.0,-1.0,-1.0,...,-1.0,1.0,45.0,572160.0,1.0,1.0,1.0,1.0,NaN,NaN


In [59]:
# inputsubmission, inputwatchlist, inputshudounyuusatsu, inputrakusatsu, inputdateorder=(train_KaiinID, train_watchlist, train_shudounyuusatsu, train_rakusatsu, dateorder-7*i)
# df = pd.concat([inputwatchlist[["KaiinID", "ShouhinID"]],
#                 inputshudounyuusatsu[["KaiinID", "ShouhinID"]]])
# df = df[df.duplicated() != True]
# df = pd.merge(df, auction[["AuctionID", "ShouhinID"]], on='ShouhinID')
# df = df[df.duplicated() != True]
# df = pd.merge(df, inputsubmission, on="KaiinID")
# auction['CreateDateOrder'] = inputdateorder-auction['CreateDateOrder']
# df = pd.merge(df, auction[["AuctionID", "ShouhinShubetsuID", "ShouhinID", "SaishuppinKaisuu",
#                            "ConditionID", "BrandID", "GenreID", "GenreGroupID", "LineID", "ColorID", "DanjobetsuID", ]])
# inputwatchlist.TourokuDateOrder = inputdateorder-inputwatchlist.TourokuDateOrder
# df = pd.merge(df, inputwatchlist.groupby(["KaiinID", "AuctionID"]).TourokuDateOrder.max(
# ).reset_index().rename(columns={'TourokuDateOrder': 'TourokuDateOrdermax'}), how='left')
# df = pd.merge(df, inputwatchlist.groupby(["KaiinID", "AuctionID"]).TourokuDateOrder.min(
# ).reset_index().rename(columns={'TourokuDateOrder': 'TourokuDateOrdermin'}), how='left')
# df = pd.merge(df, inputwatchlist.groupby(["KaiinID", "AuctionID"]).TourokuDateOrder.count(
# ).reset_index().rename(columns={'TourokuDateOrder': 'TourokuCount'}), how='left')
# df["TourokuDatemaxmindiff"] = df.TourokuDateOrdermax-df.TourokuDateOrdermin
# df = pd.merge(df, inputwatchlist.groupby(["KaiinID", "AuctionID"]).SakujoFlag.max(
# ).reset_index().rename(columns={'SakujoFlag': 'watchSakujo'}), how='left')
# Shouhincount = pd.concat([pd.Series(list(inputwatchlist.groupby("ShouhinID").ShouhinID.count(
# ).index)), pd.Series(inputwatchlist.groupby("ShouhinID").ShouhinID.count().values)], axis=1)
# Shouhincount.columns = ["ShouhinID", "ShouhinWatchedCount"]
# df = pd.merge(df, Shouhincount, how="left")
# Auctioncount = pd.concat([pd.Series(list(inputwatchlist.groupby("AuctionID").AuctionID.count(
# ).index)), pd.Series(inputwatchlist.groupby("AuctionID").AuctionID.count().values)], axis=1)
# Auctioncount.columns = ["AuctionID", "AuctionWatchedCount"]
# Auctioncount
# df = pd.merge(df, Auctioncount, how="left")
# Brandcount = pd.concat([pd.Series(list(inputwatchlist.groupby("BrandID").BrandID.count(
# ).index)), pd.Series(inputwatchlist.groupby("BrandID").BrandID.count().values)], axis=1)
# Brandcount.columns = ["BrandID", "BrandWatchedCount"]
# Brandcount
# df = pd.merge(df, Brandcount, how="left")
# Linecount = pd.concat([pd.Series(list(inputwatchlist.groupby("LineID").LineID.count(
# ).index)), pd.Series(inputwatchlist.groupby("LineID").LineID.count().values)], axis=1)
# Linecount.columns = ["LineID", "LineWatchedCount"]

# df = pd.merge(df, Linecount, how="left")
# ShouhinKaiinWatchcount = inputwatchlist.groupby(
#     ["KaiinID", "ShouhinID"]).AuctionID.count()
# ShouhinKaiinWatchcount = ShouhinKaiinWatchcount.reset_index().rename(
#     columns={"AuctionID": "ShouhinKaiinWatchCount"})
# df = pd.merge(df, ShouhinKaiinWatchcount, how="left")
# BrandKaiinWatchcount=inputwatchlist.groupby(["KaiinID","BrandID"]).AuctionID.count()
# BrandKaiinWatchcount=BrandKaiinWatchcount.reset_index().rename(columns={"AuctionID":"BrandKaiinWatchCount"})
# df=pd.merge(df,BrandKaiinWatchcount,how="left")
# GenreKaiinWatchcount=inputwatchlist.groupby(["KaiinID","GenreID"]).AuctionID.count()
# GenreKaiinWatchcount=GenreKaiinWatchcount.reset_index().rename(columns={"AuctionID":"GenreKaiinWatchCount"})
# df=pd.merge(df,GenreKaiinWatchcount,how="left")
# GenreGroupKaiinWatchcount=inputwatchlist.groupby(["KaiinID","GenreGroupID"]).AuctionID.count()
# GenreGroupKaiinWatchcount=GenreGroupKaiinWatchcount.reset_index().rename(columns={"AuctionID":"GenreGroupKaiinWatchCount"})
# df=pd.merge(df,GenreGroupKaiinWatchcount,how="left")
# inputshudounyuusatsu.ShudouNyuusatsuDateOrder=inputdateorder-inputshudounyuusatsu.ShudouNyuusatsuDateOrder
# df=pd.merge(df,inputshudounyuusatsu.groupby(["KaiinID","AuctionID"]).ShudouNyuusatsuDateOrder.max().reset_index().rename(columns={'ShudouNyuusatsuDateOrder': 'ShudouNyuusatsuDateOrdermax'}),how='left')
# df=pd.merge(df,inputshudounyuusatsu.groupby(["KaiinID","AuctionID"]).ShudouNyuusatsuDateOrder.min().reset_index().rename(columns={'ShudouNyuusatsuDateOrder': 'ShudouNyuusatsuDateOrdermin'}),how='left')
# df=pd.merge(df,inputshudounyuusatsu.groupby(["KaiinID","AuctionID"]).ShudouNyuusatsuDateOrder.count().reset_index().rename(columns={'ShudouNyuusatsuDateOrder': 'CreateCount'}),how='left')
# df["CreateDatemaxmindiff"]=df.ShudouNyuusatsuDateOrdermax-df.ShudouNyuusatsuDateOrdermin
# AuctionNyuusatsucount=pd.concat([pd.Series(list(inputshudounyuusatsu.groupby("AuctionID").AuctionID.count().index)),pd.Series(inputshudounyuusatsu.groupby("AuctionID").AuctionID.count().values)],axis=1)
# AuctionNyuusatsucount.columns=["AuctionID","AuctionNyuudstsuCount"]
# AuctionNyuusatsucount
# df=pd.merge(df,AuctionNyuusatsucount,how="left")
# KaiinNyuusatsucount=pd.concat([pd.Series(list(inputshudounyuusatsu.groupby("KaiinID").KaiinID.count().index)),pd.Series(inputshudounyuusatsu.groupby("KaiinID").KaiinID.count().values)],axis=1)
# KaiinNyuusatsucount.columns=["KaiinID","KaiinNyuudstsuCount"]
# df=pd.merge(df,KaiinNyuusatsucount,how="left")
# BrandNyuusatsucount=pd.concat([pd.Series(list(inputshudounyuusatsu.groupby("BrandID").BrandID.count().index)),pd.Series(inputshudounyuusatsu.groupby("BrandID").BrandID.count().values)],axis=1)
# BrandNyuusatsucount.columns=["BrandID","BrandNyuudstsuCount"]
# df=pd.merge(df,BrandNyuusatsucount,how="left")
# BrandNyuusatsucount=pd.concat([pd.Series(list(inputshudounyuusatsu.groupby("BrandID").BrandID.count().index)),pd.Series(inputshudounyuusatsu.groupby("BrandID").BrandID.count().values)],axis=1)
# BrandNyuusatsucount.columns=["BrandID","BrandNyuudstsuCount"]
# df=pd.merge(df,BrandNyuusatsucount,how="left")

# LineNyuusatsucount=pd.concat([pd.Series(list(inputshudounyuusatsu.groupby("LineID").LineID.count().index)),pd.Series(inputshudounyuusatsu.groupby("LineID").LineID.count().values)],axis=1)
# LineNyuusatsucount.columns=["LineID","LineNyuudstsuCount"]
# df=pd.merge(df,LineNyuusatsucount,how="left")

# ShouhinKaiinNyuusatsucount=inputshudounyuusatsu.groupby(["KaiinID","ShouhinID"]).AuctionID.count()
# ShouhinKaiinNyuusatsucount=ShouhinKaiinNyuusatsucount.reset_index().rename(columns={"AuctionID":"ShouhinKaiinNyuusatsuCount"})
# df=pd.merge(df,ShouhinKaiinNyuusatsucount,how="left")

# BrandKaiinNyuusatsucount=inputshudounyuusatsu.groupby(["KaiinID","BrandID"]).AuctionID.count()
# BrandKaiinNyuusatsucount=BrandKaiinNyuusatsucount.reset_index().rename(columns={"AuctionID":"BrandKaiinNyuusatsuCount"})
# df=pd.merge(df,BrandKaiinNyuusatsucount,how="left")

# GenreKaiinNyuusatsucount=inputshudounyuusatsu.groupby(["KaiinID","GenreID"]).AuctionID.count()
# GenreKaiinNyuusatsucount=GenreKaiinNyuusatsucount.reset_index().rename(columns={"AuctionID":"GenreKaiinNyuusatuCount"})
# df=pd.merge(df,GenreKaiinNyuusatsucount,how="left")

# LineKaiinNyuusatsucount=inputshudounyuusatsu.groupby(["KaiinID","LineID"]).AuctionID.count()
# LineKaiinNyuusatsucount=LineKaiinNyuusatsucount.reset_index().rename(columns={"AuctionID":"LineKaiinNyuusatuCount"})
# df=pd.merge(df,LineKaiinNyuusatsucount,how="left")

# df=df.drop(["ShouhinID","ShouhinShubetsuID","BrandID","GenreID","GenreGroupID","LineID"],axis=1)

# df=df.fillna(-1)

# df

/home/ogata/.pyenv/versions/3.7.0/lib/python3.7/site-packages/pandas/core/generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,KaiinID,AuctionID,SaishuppinKaisuu,ConditionID,ColorID,DanjobetsuID,TourokuDateOrdermax,TourokuDateOrdermin,TourokuCount,TourokuDatemaxmindiff,...,CreateCount,CreateDatemaxmindiff,AuctionNyuudstsuCount,KaiinNyuudstsuCount,BrandNyuudstsuCount,LineNyuudstsuCount,ShouhinKaiinNyuusatsuCount,BrandKaiinNyuusatsuCount,GenreKaiinNyuusatuCount,LineKaiinNyuusatuCount
0,231713,1036190,0,6,15,0,381.0,381.0,1.0,0.0,...,-1.0,-1.0,-1.0,45.0,319.0,572160.0,-1.0,-1.0,-1.0,41.0
1,231713,4106265,0,6,68,0,353.0,353.0,1.0,0.0,...,-1.0,-1.0,-1.0,45.0,4521.0,572160.0,-1.0,-1.0,-1.0,41.0
2,58564,4106265,0,6,68,0,358.0,358.0,1.0,0.0,...,-1.0,-1.0,-1.0,54.0,4521.0,572160.0,-1.0,-1.0,1.0,54.0
3,81233,4106265,0,6,68,0,381.0,381.0,1.0,0.0,...,-1.0,-1.0,-1.0,47.0,4521.0,572160.0,-1.0,-1.0,-1.0,47.0
4,79459,4106265,0,6,68,0,348.0,348.0,1.0,0.0,...,-1.0,-1.0,-1.0,25.0,4521.0,572160.0,-1.0,7.0,-1.0,25.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3459604,47024,2007419,0,6,297,0,-1.0,-1.0,-1.0,-1.0,...,1.0,0.0,1.0,1.0,421.0,572160.0,1.0,1.0,1.0,1.0
3459605,124155,1784243,0,6,0,0,-1.0,-1.0,-1.0,-1.0,...,1.0,0.0,1.0,1.0,182.0,572160.0,1.0,1.0,1.0,1.0
3459606,87578,2923626,0,6,68,2,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,1.0,45.0,572160.0,1.0,1.0,1.0,1.0
3459607,87578,1977247,0,6,68,2,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,1.0,45.0,572160.0,1.0,1.0,1.0,1.0
